<a href="https://colab.research.google.com/github/Last-Vega/Klis_Workshop_MachineLearning/blob/master/CJSJ_ML_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ロジスティック回帰

次に分類タスクについて，基本的なロジスティック回帰モデルを適用してみよう．

データセット（講義で説明したところの「経験」）には，乳がんデータを利用する．

## Breast cancer wisconsin dataset
https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

- 胸部腫瘍の診断画像の特徴とそれ[リンクテキスト](https://)が良性と悪性のいずれかであるかを診断した結果が含まれている
- 「サンプルデータ」として広く利用される

In [ ]:
from sklearn.datasets import load_breast_cancer # scikit-learnという機械学習のためのPythonパッケージからBreast cancer wisconsin datasetを読み込むための関数load_breast_cancerをimportする
cancer = load_breast_cancer() # load_breast_cancer関数を実行してBreast cancer wisconsin datasetを読み込み，それをcancerという変数に代入
print(cancer.DESCR) # DESCRという属性にはデータセットの詳細が書かれているので，これをprintで表示させる．

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

## データセットの中身

以下の10の特徴を複数枚の診断画像ごとに計算し，腫瘍ごとに平均，標準偏差，最大（または最小）値をとった値を腫瘍の特徴として用いている．

- 半径
- テクスチャ
- 周囲長
- 面積
- 滑らかさ
- コンパクトさ
- 凹面
- 凹点
- 対称性
- フラクタル寸法

## データセットの確認

Breast cancer wisconsin datasetでは，悪性（Malignant）と良性（Benign）の2種類のラベル（クラス）が与えられており，これらのいずれかを各患者ごとに予測する分類タスクを行う．
より正確にはあるn=30次元実数値ベクトルxにたいして0または1のいずれかを予測するというタスクである．

In [ ]:
cancer.data # dataという属性に30次元のベクトルが格納されている

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [ ]:
# 30次元のベクトルは行ベクトル（横長のベクトル）で表現されており，それが569個積み重なることによって行列として表現されている（計画行列とも呼ぶ）．つまり，569行 x 30列の行列となっている．
# この行列はnumpy.ndarrayというクラスのオブジェクトとして表現されており，shapeという属性には行と列の値が格納されている．
cancer.data.shape

(569, 30)

In [ ]:
cancer.data[0] # 試しに1行目のベクトルを取り出してみる．30次元のベクトルであることが確認できる．

array([1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
       3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
       8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
       3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
       1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01])

In [ ]:
# 悪性か良性かはtarget属性に格納されている．これは569次元のベクトルで表現され，Python中ではnumpy.ndarrayというクラスのオブジェクトとして表現されている．
# 0が悪性，1が良性である．
cancer.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [ ]:
# 計画行列の先頭の469行分のベクトルとそれに対応するラベルを訓練データ，残りをテストデータとして用いることにする
train_num = 469
train_X, test_X = cancer.data[:train_num], cancer.data[train_num:]
train_y, test_y = cancer.target[:train_num], cancer.target[train_num:]

## ロジスティック回帰モデル

ロジスティック回帰モデルはシンプルな分類モデルである．1つ前に実践したであろう線形回帰モデルと構造は非常によく似ている．単純であるが解釈性（どのように予測結果が得られたのかの理解が容易であるか）は高い．

あるn次元のベクトル${\mathbf x}$に対して，ロジスティック回帰モデルは以下の式で予測値$\hat{y}$を求める．

$$\hat{y} = {\rm argmax}_{y \in Y} P(y|{\mathbf x})$$

ただし，$Y=\{0, 1\}$であり，この式の意味は，$y=0$のときの$P(y|{\mathbf x})$，すなわち，$P(y=0|{\mathbf x})$と，$y=1$と設定した場合の$P(y|{\mathbf x})$，$P(y=1|{\mathbf x})$を比べて，前者が大きければ$\hat{y} =0$，後者が大きければ$\hat{y} =1$とするということである．

$P(y|{\mathbf x})$は，ベクトル${\mathbf x}$が与えられたときに，どれくらいの確率で$y=0$，または，$y=1$かを表すものであり，以下のように定義するのがロジスティック回帰である（つまり，他にも$P(y|{\mathbf x})$を推定する式はいろいろ考えられると言うことである）．

$$P(y=1|{\mathbf x}) = \sigma({\mathbf w}^T{\mathbf x}+b)$$

${\mathbf w}$はn次元の実数値ベクトル，$b$は実数である．
また，$P(y=0|{\mathbf x})=1-P(y=1|{\mathbf x})$と定義することにする．
このとき，$P(y=1|{\mathbf x})>P(y=0|{\mathbf x})$であることと，$P(y=1|{\mathbf x})>0.5$は同値であることがわかるであろう．
（2種類の事象の一方が起こる確率が半分以上なら，他方の起こる確率は半分以下である）

$\sigma$はシグモイド関数と呼ばれ，以下の式で定義される：

$$\sigma(x) = \frac{1}{1+\exp(-x)}$$

$P(y=1|{\mathbf x})$の式をよく見ると，シグモイド関数$\sigma$の中身は線形回帰モデルである．
では，このシグモイド関数がどのような働きをしているかというと，
実数値を$[0, 1]$の範囲に押し込んでくれるという効果がある．
${\mathbf w}^T{\mathbf x}+b$は任意の実数を取り得るが，確率はかならず$[0, 1]$という範囲の値でなければいけないため，この$\sigma$という便利な関数が用いられている．



In [ ]:
from sklearn.linear_model import LogisticRegression # scikit-learnからロジスティック回帰モデルLogisticRegressionをimportする
from sklearn.metrics import accuracy_score # 性能指標として「精度」をimportする

model = LogisticRegression() # ロジスティック回帰モデルを初期化する
trained_model = model.fit(train_X, train_y) # 訓練データ train_X, train_y を与えて，モデルを学習（＝パラメータを決定）する
predicted_y = trained_model.predict(test_X) # テストデータの事例 test_X のそれぞれに対して，予測値を計算する．
performance = accuracy_score(test_y, predicted_y) # 予測値の「精度」を求める．高い方が正確な予測だと言える．
print("精度", performance) # 「精度」を出力

精度 0.94


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## 比較の重要性

さて，また比較を行ってみよう．精度0.94（9割以上のラベルを正確に予測した）はなかなか高そうである．当てずっぽうの予測とGBDTによる予測とを比較方法としてみよう．

In [ ]:
import random
from sklearn.ensemble import GradientBoostingClassifier

# 比較手法 1
predicted_y = [random.randint(0, 1)] * len(test_X) # 0/1をランダムに予測したとする
performance = accuracy_score(test_y, predicted_y) # 予測値の精度を求める．大きい方が正確な予測だと言える．
print("精度 (Random)", performance)

# 比較手法 2
model = GradientBoostingClassifier() # GBDTモデルを初期化する
trained_model = model.fit(train_X, train_y) # 訓練データ train_X, train_y を与えて，モデルを学習する
predicted_y = trained_model.predict(test_X) # テストデータの事例 test_X のそれぞれに対して，予測値を計算する．
performance = accuracy_score(test_y, predicted_y) # 予測値の精度を求める．大きい方が正確な予測だと言える．
print("精度 (GBDT)", performance)

精度 (Random) 0.77
精度 (GBDT) 0.99


## 比較のまとめ

3つの手法を比較し以下のような結果が得られた：

- ランダム予測： 0.77
- ロジスティック回帰： 0.94
- GBDT： 0.99

ロジスティック回帰がランダムに比べると非常に良いこと，GBDTが一貫して高い性能を達成できていることがわかる．